### Homework 3 - Path + argparse + hydra

## Theory

CLI (command line interrfaces) are rarely useful without extensive and easy to use options. In this lesson, we will learn how to make good command line interfaces and store settings for them on the example of `ls` command

In [1]:
! ls --help

Usage: ls [OPTION]... [FILE]...
List information about the FILEs (the current directory by default).
Sort entries alphabetically if none of -cftuvSUX nor --sort is specified.

Mandatory arguments to long options are mandatory for short options too.
  -a, --all                  do not ignore entries starting with .
  -A, --almost-all           do not list implied . and ..
      --author               with -l, print the author of each file
  -b, --escape               print C-style escapes for nongraphic characters
      --block-size=SIZE      with -l, scale sizes by SIZE when printing them;
                               e.g., '--block-size=M'; see SIZE format below
  -B, --ignore-backups       do not list implied entries ending with ~
  -c                         with -lt: sort by, and show, ctime (time of last
                               modification of file status information);
                               with -l: show ctime and sort by name;
                               othe

In [2]:
! ls -alh HW1.ipynb HW2.ipynb

-rwx------+ 1 kirik kirik 1.2M Jul  4 00:34 HW1.ipynb
-rwx------+ 1 kirik kirik 278K Jul  8 20:47 HW2.ipynb


Pretty help may be implemented using `argparse` module.

Here is a dummy code using argparse:

In [3]:
import argparse

parser = argparse.ArgumentParser(
    description="Example",
    epilog="Text following the arguments",
)
parser.add_argument("-a", "--arg", help="some argument")
args = parser.parse_args(["-a", "val"])
parser.print_help()

usage: ipykernel_launcher.py [-h] [-a ARG]

Example

optional arguments:
  -h, --help         show this help message and exit
  -a ARG, --arg ARG  some argument

Text following the arguments


However, this is not very useful in machine learning, there it is necessary to perform a lot of runs with similar settings and save those settings alongside the results. For this purpose, the `hydra` module is more useful.

Here is its usage:

In [4]:
%%writefile config.yaml

db:
  driver: postgresql
  pass: drowssap
  timeout: 20
  user: postgres_user

Writing config.yaml


In [5]:
%%writefile hydra_example.py

import hydra
from omegaconf import DictConfig, OmegaConf

@hydra.main(version_base=None, config_path=".", config_name="config")
def main(cfg : DictConfig) -> None:
    print(OmegaConf.to_yaml(cfg))

if __name__ == "__main__":
    main()

Overwriting hydra_example.py


In [6]:
%run hydra_example.py

db:
  driver: postgresql
  pass: drowssap
  timeout: 20
  user: postgres_user



In [7]:
%run hydra_example.py --help

hydra_example is powered by Hydra.

== Configuration groups ==
Compose your configuration from those groups (group=option)

outputs/2023-07-10/10-01-51/.hydra: config, hydra, overrides


== Config ==
Override anything in the config (foo.bar=value)

db:
  driver: postgresql
  pass: drowssap
  timeout: 20
  user: postgres_user


Powered by Hydra (https://hydra.cc)
Use --hydra-help to view Hydra specific help




In [8]:
%run hydra_example.py --hydra-help

Hydra (1.3.1)
See https://hydra.cc for more info.

== Flags ==
--help,-h : Application's help
--hydra-help : Hydra's help
--version : Show Hydra's version and exit
--cfg,-c : Show config instead of running [job|hydra|all]
--resolve : Used in conjunction with --cfg, resolve config interpolations before printing.
--package,-p : Config package to show
--run,-r : Run a job
--multirun,-m : Run multiple jobs with the configured launcher and sweeper
--shell-completion,-sc : Install or Uninstall shell completion:
    Bash - Install:
    eval "$(python hydra_example.py -sc install=bash)"
    Bash - Uninstall:
    eval "$(python hydra_example.py -sc uninstall=bash)"

    Fish - Install:
    python hydra_example.py -sc install=fish | source
    Fish - Uninstall:
    python hydra_example.py -sc uninstall=fish | source

    Zsh - Install:
    Zsh is compatible with the Bash shell completion, see the [documentation](https://hydra.cc/docs/1.2/tutorials/basic/running_your_app/tab_completion#zsh-instru

Unfortunately, `hydra` has worse help compared to `argparse`. However, it allows to save settings in files, override them using commandline and perform multiruns.

In [9]:
%run hydra_example.py db.user=admin

db:
  driver: postgresql
  pass: drowssap
  timeout: 20
  user: admin



For file access, use `pathlib`, not `os`.

## Practice

Implement `myls.py` with the following options using `argparse` module:

In [10]:
%%writefile myls.py

import argparse


def main():
        # TODO: implement ls functionality

if __name__ == "__main__":
    main()

Writing myls.py


In [11]:
%run myls.py -alvv --foo HW1.ipynb HW2.ipynb

Current verbosity level: 2
Unknown args: ['--foo']
Arguments:	all: true
	files:
	- myls.py
	- myls_hydra.py
	help: false
	human_readable: false
	l: true
	recursive: false
	verbosity: 2
-rwx------+ 1 kirik kirik 1.2M Jul  4 00:34 HW1.ipynb
-rwx------+ 1 kirik kirik 278K Jul  8 20:47 HW2.ipynb


In [12]:
%%writefile myls_config.yaml

all: true
files:
- myls.py
- myls_hydra.py
- myls_config.yaml
human_readable: true
l: true
recursive: false
verbosity: 0

Writing myls_config.yaml


In [13]:
%%writefile myls_config2.yaml

all: false
files:
- foo
- bar
human_readable: false
l: false
recursive: true
verbosity: 2

Writing myls_config2.yaml


In [14]:
%%writefile myls_hydra.py

import hydra
from omegaconf import DictConfig, OmegaConf


@hydra.main(version_base=None, config_path=".", config_name="myls_config.yaml")
def main(cfg : DictConfig) -> None:
    print(cfg)
    # TODO: implement ls functionality
    # Ideally, import from myls.py file necessary function

if __name__ == "__main__":
    main()

Writing myls_hydra.py


In [15]:
%run myls_hydra.py --multirun human_readable=false,true all=false,true files=[.],[..]

[2023-07-10 10:01:53,503][HYDRA] Launching 8 jobs locally
[2023-07-10 10:01:53,503][HYDRA] 	#0 : human_readable=False all=False files=[.]
{'all': False, 'files': ['.'], 'human_readable': False, 'l': True, 'recursive': False, 'verbosity': 0}
[2023-07-10 10:01:53,611][HYDRA] 	#1 : human_readable=False all=False files=[..]
{'all': False, 'files': ['..'], 'human_readable': False, 'l': True, 'recursive': False, 'verbosity': 0}
[2023-07-10 10:01:53,710][HYDRA] 	#2 : human_readable=False all=True files=[.]
{'all': True, 'files': ['.'], 'human_readable': False, 'l': True, 'recursive': False, 'verbosity': 0}
[2023-07-10 10:01:53,812][HYDRA] 	#3 : human_readable=False all=True files=[..]
{'all': True, 'files': ['..'], 'human_readable': False, 'l': True, 'recursive': False, 'verbosity': 0}
[2023-07-10 10:01:53,967][HYDRA] 	#4 : human_readable=True all=False files=[.]
{'all': False, 'files': ['.'], 'human_readable': True, 'l': True, 'recursive': False, 'verbosity': 0}
[2023-07-10 10:01:54,079][HYD

In [1]:
#1) написать ls ??
#2) pathlib